# Orchestrating Custom Pipelines in Vertex AI

**Contents:**
1. Introduction
2. Pipeline Parameters and Variables
3. Building Execution Container
4. Setting up build triggers with Cloud Build
5. Setting up Pipeline Schedule

## 1.0 Introduction

This notebook can be used as templete to set up version control, automated build triggers and scheduling for Vertex AI and MLOps use cases. The services used are: Cloud Build, Cloud Scheduler, Artifact Registry, Cloud Run and GitHub.

In [9]:
!ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [2]:
CLIENT_SERVICE_ACCOUNT_EMAIL = "ml1-dev-sa@mlops-dev-999-c6b8.iam.gserviceaccount.com"

In [3]:
import os

PROJECT_ID = "mlops-dev-999"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  mlops-dev-999-c6b8


## 2.0 Pipeline Parameters and Variables

In [4]:
#project
PROJECT_ID = "mlops-dev-999-c6b8"
PROJECT_NUM = "965234628650"
LOCATION = "us-central1"

#artifact registry
ART_REPO_NAME = "ml-dev-99"
IMAGE_NAME = "earnings-pipline-kf"
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
IMAGE_TAG = TIMESTAMP
IMAGE_URI = "us-central1-docker.pkg.dev/{}/{}/{}:{}".format(PROJECT_ID, ART_REPO_NAME, IMAGE_NAME,IMAGE_TAG)
print(IMAGE_URI)

#container execution
URI = "https://earnings-prediction-job-znmfc5a5eq-uc.a.run.app"

#build trigger
REPO_NAME = "mlops-earnings-pipeline"
REPO_OWNER = "HenrikWarf"

#schedule
CRON = "0 9 * * *" #09.00 every day
SCHEDULE_NAME = "earnings-prediction-pipeline"


us-central1-docker.pkg.dev/mlops-dev-999-c6b8/ml-dev-99/earnings-pipline-kf:20220703142054


## 3.0 Defining Pipeline Trigger Container 

In [5]:
%%writefile Dockerfile

# Specifies base image and tag
FROM gcr.io/google.com/cloudsdktool/cloud-sdk:latest
WORKDIR /root

# Copies the trainer code to the docker image.
COPY cloud-run-execution/main.py /root/main.py
COPY pipeline-code/earnings_pipeline.json /root/earnings_pipeline.json
COPY cloud-run-execution/requirements.txt /root/requirements.txt

# Installs additional packages
RUN pip3 install -r requirements.txt 

#Execute the Application
#ENTRYPOINT ["python3", "pipeline-run.py"]
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 main:app

Overwriting Dockerfile


## 4.0 Setting up build tiggers with Cloud Build

In [9]:
%%writefile cloud-build.yaml

steps: 
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'us-central1-docker.pkg.dev/${PROJECT_ID}/ml-dev-99/earnings-pipline-kf:${BUILD_ID}', '.']
  id: 'Building Cloud Run Container - Pipeline Execution Step'

- name: 'gcr.io/cloud-builders/docker'
  args: [ 'push', 'us-central1-docker.pkg.dev/${PROJECT_ID}/ml-dev-99/earnings-pipline-kf:${BUILD_ID}']
  id: 'Pushing Cloud Run Container to Artifact Registrty'

- name: 'gcr.io/cloud-builders/gcloud'
  args: 
    - 'run'
    - 'deploy'
    - 'earnings-prediction-job'
    - '--region'
    - 'us-central1'
    - '--image'
    - 'us-central1-docker.pkg.dev/${PROJECT_ID}/ml-dev-99/earnings-pipline-kf:${BUILD_ID}'
    - '--service-account'
    - 'ml1-dev-sa@mlops-dev-999-c6b8.iam.gserviceaccount.com'
  id: 'Deploy Pipeline Execution container in Cloud Run'

Overwriting cloud-build.yaml


# 5.0 Set up Artifact Registry

In [32]:
!gcloud artifacts repositories create ml-dev-99 --repository-format=docker \
    --location=us-central1 --description="Docker repository for ml-dev-99"

Create request issued for: [ml-dev-99]
Waiting for operation [projects/mlops-dev-999-c6b8/locations/us-central1/operat
ions/56180a11-cd41-4410-8fd0-944eb99107cc] to complete...done.                 
Created repository [ml-dev-99].


# 6.0 Set up build trigger connection to git repo

In [10]:
!gcloud config set project --quiet mlops-dev-999-c6b8

Updated property [core/project].


In [7]:
!gcloud config list

[core]
account = admin@henrikw.altostrat.com
disable_usage_reporting = True
project = mlops-dev-999-c6b8

Your active configuration is: [default]


In [15]:
!gcloud auth list

Credentialed Accounts

ACTIVE: *
ACCOUNT: admin@henrikw.altostrat.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [14]:
! echo $PROJECT_ID

mlops-dev-999-c6b8


In [13]:
PROJECT_ID

'mlops-dev-999-c6b8'

In [16]:
!gcloud beta builds triggers create github --quiet --name=$REPO_NAME\
--project=$PROJECT_ID\
--repo-owner=$REPO_OWNER\
--repo-name=$REPO_NAME\
--branch-pattern=".*"\
--region="us-central1"\
--build-config="cloud-build.yaml"

Created [https://cloudbuild.googleapis.com/v1/projects/mlops-dev-999-c6b8/locations/us-central1/triggers/46527960-7fe8-4f39-a123-6ef28779cdd6].
NAME: mlops-earnings-pipeline
CREATE_TIME: 2022-07-03T12:13:09+00:00
STATUS: 


#### Deleting the build trigger

In [12]:
!gcloud beta builds triggers delete mlops-earnings-pipeline --region="us-central1" 

Deleted [projects/mlops-dev-999-c6b8/locations/us-central1/triggers/mlops-earnings-pipeline].


#### Manual Trigger

In [ ]:
!gcloud builds submit --config cloud-build.yaml .

In [ ]:
!gcloud builds submit --region=us-west2 --tag gcr.io/project-id/image-name .

## 5.0 Setting up Pipeline Schedule

#### Set up schedule

In [4]:
URI

'https://earnings-prediction-job-znmfc5a5eq-uc.a.run.app'

In [8]:
!gcloud scheduler jobs create http $SCHEDULE_NAME\
--project="mlops-dev-999-c6b8" --location="us-central1" --schedule="0 */3 * * *"\
--uri=$URI --http-method=POST --oidc-service-account-email=$CLIENT_SERVICE_ACCOUNT_EMAIL

#### Delete Schedule

In [7]:
!gcloud scheduler jobs delete $SCHEDULE_NAME --location=$LOCATION --quiet

Deleted job [earnings-prediction-pipeline].
